In [2]:
import os
import cv2
import logging

# Set up logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s [%(levelname)s] %(message)s")

# Define the source and destination folders
source_folder = r'C:\Users\user\Desktop\photos'  # Replace with your "photos" folder path
destination_folder = r'C:\Users\user\Desktop\gray'  # Replace with your "gray" folder path

# Create the destination folder if it doesn't exist
if not os.path.exists(destination_folder):
    os.makedirs(destination_folder)
    logging.info(f"Created destination folder: {destination_folder}")

# Process each image in the source folder
for image_name in os.listdir(source_folder):
    source_path = os.path.join(source_folder, image_name)
    destination_path = os.path.join(destination_folder, image_name)

    # Check if the file is an image
    if os.path.isfile(source_path) and image_name.lower().endswith(('.jpg', '.jpeg', '.png')):
        try:
            # Read the image
            image = cv2.imread(source_path)
            if image is None:
                logging.warning(f"Could not read image: {source_path}")
                continue

            # Convert the image to grayscale
            gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

            # Save the grayscale image to the destination folder
            cv2.imwrite(destination_path, gray_image)
            logging.info(f"Converted and saved: {destination_path}")

        except Exception as e:
            logging.error(f"Error processing {image_name}: {e}")
    else:
        logging.warning(f"Skipped non-image file: {source_path}")

logging.info("Image conversion process completed.")


2025-01-19 10:10:33,693 [INFO] Converted and saved: C:\Users\user\Desktop\gray\aradhya.jpg
2025-01-19 10:10:33,704 [INFO] Converted and saved: C:\Users\user\Desktop\gray\ashish.jpg
2025-01-19 10:10:33,715 [INFO] Converted and saved: C:\Users\user\Desktop\gray\ayush.jpg
2025-01-19 10:10:34,082 [INFO] Converted and saved: C:\Users\user\Desktop\gray\bora.jpg
2025-01-19 10:10:34,110 [INFO] Converted and saved: C:\Users\user\Desktop\gray\devendra.jpg
2025-01-19 10:10:34,184 [INFO] Converted and saved: C:\Users\user\Desktop\gray\dhirendra.jpg
2025-01-19 10:10:34,317 [INFO] Converted and saved: C:\Users\user\Desktop\gray\harshit.jpg
2025-01-19 10:10:34,995 [INFO] Converted and saved: C:\Users\user\Desktop\gray\kartik.jpg
2025-01-19 10:10:35,025 [INFO] Converted and saved: C:\Users\user\Desktop\gray\manoj.jpg
2025-01-19 10:10:35,060 [INFO] Converted and saved: C:\Users\user\Desktop\gray\Mr. Vinod.jpg
2025-01-19 10:10:35,113 [INFO] Converted and saved: C:\Users\user\Desktop\gray\nikhil.jpg
2025

In [7]:
import cv2
import face_recognition
import os

# Path to the folder containing known face images
known_faces_folder = r'C:\Users\user\Desktop\gray'  # Replace with your folder path

# Initialize lists for known face encodings and names
known_face_encodings = []
known_face_names = []

# Load each image from the folder and create encodings
for image_file in os.listdir(known_faces_folder):
    if image_file.endswith(('.jpg', '.jpeg', '.png')):  # Check for image file types
        image_path = os.path.join(known_faces_folder, image_file)
        known_image = face_recognition.load_image_file(image_path)
        known_encoding = face_recognition.face_encodings(known_image)
        if known_encoding:  # Check if encoding was found
            known_face_encodings.append(known_encoding[0])
            known_face_names.append(os.path.splitext(image_file)[0])  # Use the filename as the name

# Start webcam
cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print("Error: Could not access the webcam.")
    exit()

while True:
    ret, frame = cap.read()
    if not ret:
        print("Failed to grab frame.")
        break

    # Flip the frame horizontally (mirror effect)
    frame = cv2.flip(frame, 1)

    # Convert to RGB for face_recognition
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    face_locations = face_recognition.face_locations(rgb_frame)
    face_encodings = face_recognition.face_encodings(rgb_frame, face_locations)

    # Recognize faces
    for (top, right, bottom, left), face_encoding in zip(face_locations, face_encodings):
        matches = face_recognition.compare_faces(known_face_encodings, face_encoding, tolerance=0.5)
        name = "Unknown"
        if True in matches:
            first_match_index = matches.index(True)
            name = known_face_names[first_match_index]

        # Draw rectangle and name
        cv2.rectangle(frame, (left, top), (right, bottom), (0, 255, 0), 2)
        cv2.putText(frame, name, (left, top - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0, 255, 0), 2)

    # Display frame
    cv2.imshow("Webcam", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):  # Press 'q' to quit
        break

cap.release()
cv2.destroyAllWindows()


In [15]:
import os
import face_recognition

# Test dataset folder path
test_images_folder = r'C:\Users\user\Desktop\gray'  # Replace with your test dataset folder

# Initialize variables
correct_predictions = 0
total_predictions = 0

# Dynamically create ground truth labels from the folder
# Assumes the file name (without extension) is the person's name
ground_truth_labels = {
    file_name: os.path.splitext(file_name)[0]
    for file_name in os.listdir(test_images_folder)
    if file_name.endswith(('.jpg', '.jpeg', '.png'))  # Supported image types
}

# Process each file in the folder
for image_file, true_label in ground_truth_labels.items():
    image_path = os.path.join(test_images_folder, image_file)

    # Check if the file exists
    if not os.path.exists(image_path):
        print(f"File not found: {image_path}")
        continue

    test_image = face_recognition.load_image_file(image_path)
    face_encodings = face_recognition.face_encodings(test_image)

    if face_encodings:
        test_encoding = face_encodings[0]
        matches = face_recognition.compare_faces(known_face_encodings, test_encoding, tolerance=0.5)
        predicted_label = "Unknown"
        if True in matches:
            first_match_index = matches.index(True)
            predicted_label = known_face_names[first_match_index]

        # Compare predicted and true labels
        if predicted_label == true_label:
            correct_predictions += 1
        total_predictions += 1

# Calculate accuracy
accuracy = (correct_predictions / total_predictions) * 100 if total_predictions > 0 else 0
print(f"Accuracy: {accuracy:.2f}%")


Accuracy: 100.00%
